Phase 1 – Implementation of the Sentiment Analysis Model

In this phase, the pre-trained model
cardiffnlp/twitter-roberta-base-sentiment-latest (based on RoBERTa)
was used to analyze texts collected from social media.

The chosen dataset is TweetEval (sentiment) from HuggingFace, which contains tweets
labeled as positive, neutral, or negative.

After a light text preprocessing step, the model was tested on the test split.
The achieved accuracy is approximately 0.68, which is consistent with the expected performance of this model.

In [2]:
import sys
import os
import torch
import evaluate

# Add project root (one level up) to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification,  AutoConfig
from src.preprocess import preprocess_texts


c:\Users\cogli\Desktop\sentiment-monitoring-mlops\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
device = 0 if torch.cuda.is_available() else -1

In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, use_safetensors=True)
config = AutoConfig.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
dataset = load_dataset("tweet_eval", "sentiment")

def preprocess_batch(batch):
    batch["text"] = preprocess_texts(batch["text"])
    return batch

cleaned_ds = dataset.map(preprocess_batch, batched=True)

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
test_ds = cleaned_ds["test"]
texts = list(test_ds["text"])
true_labels = list(test_ds["label"])

In [22]:
print(len(test_ds))

2


In [11]:
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
preds = sentiment_task(texts)

Device set to use cuda:0


In [15]:
from sklearn.metrics import accuracy_score

pred_labels = [pred["label"] for pred in preds]
label_map = {"negative": 0, "neutral": 1, "positive": 2}

pred_encoded = [label_map[p_lab] for p_lab in pred_labels]
print("Accuracy:", accuracy_score(true_labels, pred_encoded))


Accuracy: 0.6838977531748616
